In [7]:
# Import webdriver from selenium
from selenium import webdriver
chrome_path = r'C:/Users/b0130/Downloads/chromedriver_win32/chromedriver'
# prepend it with r because of the backslash characters

In [8]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd

In [9]:
def get_date(soup):
    date = []
    for element in soup.find_all(class_='MItem DateCreated'):
        date.append(element.find('time').text.strip())
    return date

In [10]:
def get_userid(soup):
    userid = []
    for element in soup.find_all(class_='Comment'):
        userid.append(element.find_all('span')[0].text.strip())
    return userid

In [11]:
def remove_quote(soup):
    cleantext = []
    text = soup.find_all('div',{'class':'Message'})
    for t in text:
        if t.find('blockquote'):
            try:
                cleantext.append(t.contents[2].strip())
            except:
                cleantext.append(u"")
        else:
            cleantext.append(t.contents[0].strip()) 
    return cleantext

In [22]:
text

[<div class="Message userContent">
                             If they keep it around in next four-five years, I think I will make myself to put on a hazmat suit and visit our friendly Kia dealer (oh, boy are they horrible here, or what) to check it out.  It's got a lot of good stuff, but seems like they are still behind on a few things, such as relationship between power and gas mileage (e.g. BMW 340/440 has similar performance, but much better gas mileage as a daily driver), but with appropriate price difference, those objections and shortcomings are not insurmountable.  Biggest thing will be the "first contact" with a sales person.  Hope it won't start from "are you buying it today?", "what can I do to make you take it home?", or "let me wash your BMW - oh, I can't find the keys", "How about I show you the deal - square one, two, three, four".                        </div>,
 <div class="Message userContent">
                             The lease rate is the factor that stops me co

In [14]:
def get_data (url, driver, pages):
    date = []
    userid = []
    cleantext = []
    for i in range (pages):
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        date += get_date(soup)
        userid += get_userid(soup)
        cleantext += remove_quote(soup)
        next_button = soup.find("a", class_="Previous Pager-nav")
        url = next_button['href']
    return pd.DataFrame(list(zip(date, userid, cleantext)))

In [18]:
url = "https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702"
driver = webdriver.Chrome(chrome_path)
data = get_data(url, driver, 170)

In [20]:
data.to_csv('final.csv', encoding='utf-8')